In [1]:
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [2]:
from model import create_model
from align import AlignDlib
import os
import numpy as np
import cv2
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
import sys
from PIL import Image
sys.modules['Image'] = Image
from PIL import Image
import warnings
from scipy import ndimage
warnings.filterwarnings('ignore')

In [3]:
# Model for 128 embeddings from a image
nn4_small2_pretrained = create_model()
nn4_small2_pretrained.load_weights('weights/nn4.small2.v1.h5')

In [4]:
# Model Summary
nn4_small2_pretrained.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 96, 96, 3)    0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 102, 102, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 48, 48, 64)   9472        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
bn1 (BatchNormalization)        (None, 48, 48, 64)   256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [5]:
# Initialize the OpenFace face alignment utility
alignment = AlignDlib('weights/landmarks.dat')

In [6]:
# Funtion to align an image using dlib utility
def align_image(img):
    return alignment.align(96, img, alignment.getLargestFaceBoundingBox(img), 
                           landmarkIndices=AlignDlib.OUTER_EYES_AND_NOSE)

In [7]:
def load_image(path):
    img = cv2.imread(path, 1)
    return img[...,::-1]

In [8]:
class IdentityMetadata():
    def __init__(self, base, name, file):
        self.base = base
        self.name = name
        self.file = file

    def __repr__(self):
        return self.image_path()
    
    def subdir_path(self):
        return self.name
    
    def image_path(self):
        return os.path.join(self.base, self.name, self.file) 
    
def load_metadata(path):
    metadata = []
    for i in sorted(os.listdir(path)):
        for f in sorted(os.listdir(os.path.join(path, i))):
            # Check file extension. Allow only jpg/jpeg' files.
            ext = os.path.splitext(f)[1]
            if ext == '.jpg' or ext == '.jpeg':
                metadata.append(IdentityMetadata(path, i, f))
    return np.array(metadata)

metadata = load_metadata('images')

In [9]:
datagen = ImageDataGenerator(rotation_range=15, 
                             width_shift_range=0.1, 
                             height_shift_range=0.1,
                             shear_range=0.15, 
                             zoom_range=0.1,
                             channel_shift_range = 10, 
                             brightness_range=[0.2,1.0],
                             horizontal_flip=True)
import os

folder_name='AugmentedImages/'
save_here = folder_name

if not os.path.exists(save_here):
    os.makedirs(save_here) 

    for i, m in enumerate(metadata): 
        img = load_image(m.image_path())
        datagen.fit(np.expand_dims(img,0))
        directory=save_here+m.subdir_path()
        if not os.path.exists(directory):
            os.makedirs(directory)
            print(directory)
        for x, val in zip(datagen.flow(np.expand_dims(img,0), 
                                       save_to_dir=directory, 
                                       save_prefix='AugImage', 
                                       save_format='jpg'),range(3)):
            pass

In [10]:
metadata1 = load_metadata('AugmentedImages')

In [11]:
# To store image embeddings
embedded = np.zeros((metadata1.shape[0], 128))

# Process image through the neural net to get 128 embeddings per image
for i, m in enumerate(metadata1):    
    print(i, " ",m)
    img = load_image(m.image_path())
    img = align_image(img)
    if img is None:
        print("No face found", m)
        continue
    # scale RGB values to interval [0,1]
    img = (img / 255.).astype(np.float32)
    # obtain embedding vector for image
    embedded[i] = nn4_small2_pretrained.predict(np.expand_dims(img, axis=0))[0]

0   AugmentedImages\Aastha\AugImage_0_1124.jpg
1   AugmentedImages\Aastha\AugImage_0_1217.jpg
2   AugmentedImages\Aastha\AugImage_0_1301.jpg
3   AugmentedImages\Aastha\AugImage_0_2024.jpg
4   AugmentedImages\Aastha\AugImage_0_2209.jpg
5   AugmentedImages\Aastha\AugImage_0_2364.jpg
6   AugmentedImages\Aastha\AugImage_0_2642.jpg
7   AugmentedImages\Aastha\AugImage_0_2946.jpg
8   AugmentedImages\Aastha\AugImage_0_2963.jpg
9   AugmentedImages\Aastha\AugImage_0_3050.jpg
10   AugmentedImages\Aastha\AugImage_0_3067.jpg
11   AugmentedImages\Aastha\AugImage_0_3214.jpg
12   AugmentedImages\Aastha\AugImage_0_357.jpg
13   AugmentedImages\Aastha\AugImage_0_3932.jpg
14   AugmentedImages\Aastha\AugImage_0_400.jpg
15   AugmentedImages\Aastha\AugImage_0_4694.jpg
16   AugmentedImages\Aastha\AugImage_0_4814.jpg
17   AugmentedImages\Aastha\AugImage_0_5152.jpg
18   AugmentedImages\Aastha\AugImage_0_5331.jpg
19   AugmentedImages\Aastha\AugImage_0_5414.jpg
20   AugmentedImages\Aastha\AugImage_0_5709.jpg
21  

162   AugmentedImages\Unknown\AugImage_0_7863.jpg
163   AugmentedImages\Unknown\AugImage_0_8053.jpg
164   AugmentedImages\Unknown\AugImage_0_8101.jpg
165   AugmentedImages\Unknown\AugImage_0_8166.jpg
166   AugmentedImages\Unknown\AugImage_0_831.jpg
167   AugmentedImages\Unknown\AugImage_0_8404.jpg
168   AugmentedImages\Unknown\AugImage_0_842.jpg
169   AugmentedImages\Unknown\AugImage_0_8478.jpg
170   AugmentedImages\Unknown\AugImage_0_8603.jpg
171   AugmentedImages\Unknown\AugImage_0_8693.jpg
172   AugmentedImages\Unknown\AugImage_0_8722.jpg
173   AugmentedImages\Unknown\AugImage_0_8764.jpg
174   AugmentedImages\Unknown\AugImage_0_883.jpg
175   AugmentedImages\Unknown\AugImage_0_8985.jpg
176   AugmentedImages\Unknown\AugImage_0_9110.jpg
177   AugmentedImages\Unknown\AugImage_0_9142.jpg
178   AugmentedImages\Unknown\AugImage_0_9189.jpg
179   AugmentedImages\Unknown\AugImage_0_9204.jpg
180   AugmentedImages\Unknown\AugImage_0_9223.jpg
181   AugmentedImages\Unknown\AugImage_0_9227.jpg
182

In [20]:
import pickle
embedding_file = open('embedding_dump', 'wb')
pickle.dump(embedded, embedding_file)
embedding_file.close()

In [ ]:
import pickle
embedding_file = open('embedding_dump', 'rb')
embedded = pickle.load(embedding_file)

In [15]:
embedded

array([[ 0.199581  , -0.06013365,  0.01814981, ..., -0.04393851,
         0.08994129, -0.01555431],
       [ 0.17890118, -0.02282499, -0.03210591, ...,  0.01720419,
         0.01596251,  0.04336134],
       [ 0.19530642, -0.04951643,  0.01249343, ..., -0.03829554,
         0.08512429, -0.011032  ],
       ...,
       [ 0.02419946,  0.08636332, -0.0468494 , ...,  0.11219005,
         0.03366694, -0.04650257],
       [ 0.07926922,  0.16669893,  0.04124511, ..., -0.00830263,
         0.17524371,  0.03113581],
       [ 0.01305444,  0.1754051 ,  0.05407042, ..., -0.12808435,
         0.10033607,  0.01638264]])

In [12]:
from sklearn.preprocessing import LabelEncoder
targets = np.array([m.name for m in metadata1])
encoder = LabelEncoder()
encoder.fit(targets)

# Numerical encoding of identities
y = encoder.transform(targets)

In [13]:
train_idx = np.arange(metadata1.shape[0]) % 2 != 0
test_idx = np.arange(metadata1.shape[0]) % 2 == 0

# 50 train examples of 10 identities (5 examples each)
X_train = embedded[train_idx]
# 50 test examples of 10 identities (5 examples each)
X_test = embedded[test_idx]

y_train = y[train_idx]
y_test = y[test_idx]

In [45]:
### SUPPORT VECTOR CLASSIFIER
# svc = SVC(kernel="linear", C=0.025) #10 -> 57% accuracy, 30-> 31%, 50->45%
# svc = SVC(gamma=2, C=1, probability=True) #30-> 88%, 50->88%
svc = SVC(kernel="rbf", C=30, probability=True)

### Gaussian CLASSIFIER
# svc = GaussianProcessClassifier(1.0 * RBF(1.0)) # 30->90%, 50->89%

### KNN CLASSIFIER
# svc = KNeighborsClassifier(2) #10 images -> 85% accuracy, 30,50 images->88%
# svc = KNeighborsClassifier(5) #10 -> 78% accuracy, 30-> 86%, 50->87%

### DECISION TREE CLASSIFIER
# svc = DecisionTreeClassifier(max_depth=5) # 30,50->58%
# svc = DecisionTreeClassifier(max_depth=10) # 30->68%, 50->79%
# svc = DecisionTreeClassifier(max_depth=20) #30-> 70%, 50->80%
# svc = DecisionTreeClassifier(max_depth=30) # 30->73%, 50->80%
# svc = DecisionTreeClassifier(max_depth=40) # 30->73%, 50->78%

### Random Forest Classifier
# svc = RandomForestClassifier(max_depth=10, n_estimators=50, max_features=1)  #10 -> 80% accuracy aprox, 30,50 -> 88%
# svc = RandomForestClassifier(40)  #10 -> 80% accuracy aprox, 30,50 -> 89%
# svc = RandomForestClassifier(30)  #10 -> 70% accuracy aprox, 30,50 -> 89%
# svc = RandomForestClassifier(20)  #10 -> 65% accuracy aprox, 30,50 -> 87%

### MLP CLASSIFIER
# svc = MLPClassifier(alpha=1.0, max_iter=1000) # 30,50->88%

### ADA BOOST CLASSIFIER
# svc = AdaBoostClassifier(n_estimators=100, random_state=0) # 30->35%, 50->28%

### QUADRATIC CLASSIFIER
# svc = QuadraticDiscriminantAnalysis() #30->40%, 50->9%

#Naive bayes algorithm
# svc = GaussianNB() #10 -> 73% accuracy aprox, 30 -> 86%, 50 -> 88%

svc.fit(X_train, y_train)

acc_svc = accuracy_score(y_test, svc.predict(X_test))

print(f'SVM accuracy = {acc_svc}')

SVM accuracy = 0.9574468085106383


In [46]:
# train_idx = np.arange(metadata.shape[0]) 

# # 50 train examples of 10 identities (5 examples each)
# X_train = embedded[train_idx]

# y_train = y[train_idx]

# # svc = KNeighborsClassifier()
# # svc = SVC()
# svc = RandomForestClassifier(20)
# svc.fit(X_train, y_train)


In [47]:
# Prediction Code for live Images

In [48]:
font = cv2.FONT_HERSHEY_SIMPLEX

def processFrame(img):
    # Change RGB to BGR for imshow
    origImg = img[...,::-1]
    
    # Get bounding box
    bb = alignment.getLargestFaceBoundingBox(img)
    
    # Align image
    img = align_image(img)
    
    # scale RGB values to interval [0,1]
    img = (img / 255.).astype(np.float32)
    
    # obtain embedding vector for image
    features = nn4_small2_pretrained.predict(np.expand_dims(img, axis=0))[0]
    
    # Get the class for prediction
    predClass = svc.predict([features])
    predClass = encoder.inverse_transform(predClass)[0]
    
    # Display the image with bounding box & label
    cv2.putText(origImg,predClass,(bb.left(),bb.top()-5), font, 0.5, (0,0,255), 1, cv2.LINE_AA)
    cv2.rectangle(origImg,(bb.left(),bb.top()),(bb.right(),bb.bottom()),(0,0,255),2)
    cv2.imshow('image',origImg)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [49]:
font = cv2.FONT_HERSHEY_SIMPLEX

def processFrame1(img):
    # Change RGB to BGR for imshow
    origImg = img[...,::-1]
    
    # Get bounding box
    bb = alignment.getLargestFaceBoundingBox(img)
    
    # Align image
    img = align_image(img)
    
    # scale RGB values to interval [0,1]
    img = (img / 255.).astype(np.float32)
    
    # obtain embedding vector for image
    features = nn4_small2_pretrained.predict(np.expand_dims(img, axis=0))[0]
    
    # Get the class for prediction
    predScores = svc.predict_proba([features])
    print(predScores)
    predClass = None
    
    maxProb = (predScores[0]).max()
    
    print(maxProb)
    
    if(maxProb >= 0.75):
    
        predClass = [(predScores[0]).argmax()]
        print(predClass)
        predClass = encoder.inverse_transform(predClass)[0]
    
    
    print(predClass)
    
    return bb, predClass, maxProb

In [ ]:
students = []

cap = cv2.VideoCapture(1)
counter = 0
studentList = []
try:
    while(True):
        counter+=1
        # Capture frame-by-frame
        ret, frame = cap.read()
    #     print(ret, " ", frame)
        if frame is not None:
            if(alignment.getLargestFaceBoundingBox(frame)!=None):
                bb, predClass, maxProb = processFrame1(frame)
                print("oo: ",predClass)
                studentList.append(predClass)
                cv2.rectangle(frame,(bb.left(),bb.top()),(bb.right(),bb.bottom()),(0,0,255),2)
                label = "" if predClass is None else predClass + " " + str(maxProb)
                print("oldLabel:", label)
                label+= " "
                label+= str(maxProb)
                cv2.putText(frame,label,(bb.left(),bb.top()-5), font, 0.5, (0,0,255), 1, cv2.LINE_AA)
                if counter==10:
                    predClass = str(max(set(studentList), key = studentList.count)) 
                    if predClass != 'None':
                        if predClass not in students:
                              students.append(predClass)
                        cv2.putText(frame,predClass,(bb.left(),bb.top()-5), font, 0.5, (0,0,255), 1, cv2.LINE_AA)
                    counter = 0
                    studentList = []
    #             if(predClass!=None):
    #                 cv2.putText(frame,predClass,(bb.left(),bb.top()-5), font, 0.5, (0,0,255), 1, cv2.LINE_AA)
    #                 if predClass not in students:
    #                     students.append(predClass)
            cv2.imshow('frame',frame)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
    #             cv2.destroyAllWindows()
except Exception as e: 
    print(e)
    cap.release()
    cv2.destroyAllWindows()

    
    
# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()
print(students)

[[0.76469032 0.0393617  0.19594798]]
0.764690317248828
[0]
Aastha
oo:  Aastha
oldLabel: Aastha 0.764690317248828
[[2.73475358e-03 9.26587769e-04 9.96338659e-01]]
0.9963386586501811
[2]
Unknown
oo:  Unknown
oldLabel: Unknown 0.9963386586501811
[[0.06003896 0.00331989 0.93664115]]
0.9366411486393943
[2]
Unknown
oo:  Unknown
oldLabel: Unknown 0.9366411486393943
[[0.07653778 0.00469485 0.91876738]]
0.9187673757923384
[2]
Unknown
oo:  Unknown
oldLabel: Unknown 0.9187673757923384
[[0.08132716 0.00542277 0.91325007]]
0.9132500731192406
[2]
Unknown
oo:  Unknown
oldLabel: Unknown 0.9132500731192406
[[0.0658986  0.0044537  0.92964771]]
0.9296477081094704
[2]
Unknown
oo:  Unknown
oldLabel: Unknown 0.9296477081094704
[[0.07811974 0.00496472 0.91691554]]
0.9169155374146114
[2]
Unknown
oo:  Unknown
oldLabel: Unknown 0.9169155374146114
[[0.04666633 0.00296    0.95037366]]
0.9503736634105829
[2]
Unknown
oo:  Unknown
oldLabel: Unknown 0.9503736634105829
[[0.27631604 0.04473423 0.67894972]]
0.6789497236

[[0.14859702 0.05852133 0.79288165]]
0.7928816481999852
[2]
Unknown
oo:  Unknown
oldLabel: Unknown 0.7928816481999852
[[0.12907373 0.05283598 0.8180903 ]]
0.8180902965252574
[2]
Unknown
oo:  Unknown
oldLabel: Unknown 0.8180902965252574
[[0.05144459 0.46137379 0.48718163]]
0.48718162682087995
None
oo:  None
oldLabel: 
[[0.03811993 0.60093738 0.36094269]]
0.6009373763688928
None
oo:  None
oldLabel: 
[[0.0495933  0.43216458 0.51824213]]
0.5182421277497147
None
oo:  None
oldLabel: 
[[0.05524556 0.51248526 0.43226918]]
0.5124852614469424
None
oo:  None
oldLabel: 
[[0.04365619 0.65240698 0.30393683]]
0.6524069788560157
None
oo:  None
oldLabel: 
[[0.04872034 0.55404492 0.39723474]]
0.5540449200805677
None
oo:  None
oldLabel: 
[[0.05561127 0.46815389 0.47623484]]
0.4762348431930946
None
oo:  None
oldLabel: 
[[0.05451197 0.53394138 0.41154665]]
0.5339413826640901
None
oo:  None
oldLabel: 
[[0.04622443 0.44355517 0.5102204 ]]
0.510220401667747
None
oo:  None
oldLabel: 
[[0.05873125 0.55708154 0.

In [20]:
str(maxProb)

'0.7604567988588229'

In [21]:
cv2.imshow("f",frame)
cv2.waitKey(3000)
cv2.destroyAllWindows()

In [17]:
import cv2
cap = cv2.VideoCapture("test.webm")
counter = 0
try:
    while True:
        ret, frame = cap.read()
        cv2.imshow('frame',frame)
        print(counter,end=" ")
        counter+=1;
        if cv2.waitKey(20) & 0xFF == ord('q'):
            break
except:        
    cap.release()
    cv2.destroyAllWindows()
cap.release()
cv2.destroyAllWindows()

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 